In [10]:
spatial_extent = {'west': 4.45, 'east': 4.50, 'south': 51.16, 'north': 51.17, 'crs': 'epsg:4326'}

In [3]:
import openeo
from openeo.api.process import Parameter
import xarray
import numpy as np
import io
import requests

import panel as pn

import pyproj
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

In [5]:
import requests
requests.packages.urllib3.util.connection.HAS_IPV6 = False

In [6]:
c=openeo.connect("openeo-dev.vito.be")
c.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo-dev.vito.be/openeo/1.1/' with OidcBearerAuth>

In [7]:
import scipy
import numpy as np

scl = c.load_collection(
    "TERRASCOPE_S2_TOC_V2",
    temporal_extent = ["2022-07-01", "2022-08-01"],
    bands = ["SCL"],
    max_cloud_cover=70
)#.resample_spatial(resolution=60)

def makekernel(iwindowsize):
    kernel_vect = scipy.signal.windows.gaussian(iwindowsize, std=iwindowsize / 6.0, sym=True)
    kernel = np.outer(kernel_vect, kernel_vect)
    kernel = kernel / kernel.sum()
    return kernel

classification = scl.band("SCL")
binary = (classification == 3) | (classification == 8) | (classification == 9) | (classification == 10) | (classification == 11)
    
distance = binary.apply_kernel(makekernel(11))
#distance.filter_bbox(spatial_extent).download("distance.nc")

openEO masking: pixels that are set to 1 will become nodata after masking.
See:
https://processes.openeo.org/#mask

"pixels in data are replaced whose pixels in mask are non-zero (for numbers) or true (for boolean values)"

In [8]:
from openeo.processes import absolute, int,inspect,multiply,array_apply

label = Parameter('label')
parent = Parameter('parent.data')

def bap_selection(ndvi):    
    scoring_rule = lambda x:absolute(15*x) + absolute(x.process("date_difference",date1=x.process("date_replace_component",date=label,value=15,component="day"),date2=label,unit="day"))
    score = ndvi.array_apply(scoring_rule)        
    return score.array_apply(lambda x:int(x)!=int(array_apply(parent,scoring_rule).min()))
    

rank_mask = distance.apply_neighborhood(
        bap_selection,
        size=[{'dimension': 'x', 'unit': 'px', 'value': 1}, {'dimension': 'y', 'unit': 'px', 'value': 1},
              {'dimension': 't', 'value': "month"}],
        overlap=[]
    )

/home/driesj/python/openeo-python-client/openeo/api/process.py:22: UserWarning: Parameter without description: using name as description.
  warnings.warn("Parameter without description: using name as description.")


In [16]:
distance.reduce_dimension("t",bap_selection).filter_bbox(spatial_extent).download("min.nc")

In [34]:
rank_mask.filter_bbox(spatial_extent).execute_batch("cloud_mask_both_fullres2.nc")

0:00:00 Job 'j-231024021a474ce8b196f88df4058efc': send 'start'
0:00:24 Job 'j-231024021a474ce8b196f88df4058efc': queued (progress N/A)
0:00:29 Job 'j-231024021a474ce8b196f88df4058efc': queued (progress N/A)
0:00:35 Job 'j-231024021a474ce8b196f88df4058efc': queued (progress N/A)
0:00:43 Job 'j-231024021a474ce8b196f88df4058efc': queued (progress N/A)
0:00:53 Job 'j-231024021a474ce8b196f88df4058efc': queued (progress N/A)
0:01:06 Job 'j-231024021a474ce8b196f88df4058efc': queued (progress N/A)
0:01:21 Job 'j-231024021a474ce8b196f88df4058efc': queued (progress N/A)
0:01:40 Job 'j-231024021a474ce8b196f88df4058efc': queued (progress N/A)
0:02:05 Job 'j-231024021a474ce8b196f88df4058efc': queued (progress N/A)
0:02:35 Job 'j-231024021a474ce8b196f88df4058efc': running (progress N/A)
0:03:12 Job 'j-231024021a474ce8b196f88df4058efc': running (progress N/A)
0:04:00 Job 'j-231024021a474ce8b196f88df4058efc': running (progress N/A)
0:04:58 Job 'j-231024021a474ce8b196f88df4058efc': finished (progress N

<BatchJob job_id='j-231024021a474ce8b196f88df4058efc'>

In [11]:
composite = c.load_collection(
    "TERRASCOPE_S2_TOC_V2",
    temporal_extent = ["2022-07-01", "2022-08-01"],
    bands = ["B02","B03"],
    max_cloud_cover=70
)
composite.mask(rank_mask).min_time().filter_bbox(spatial_extent).execute_batch("bap_composite.nc")

0:00:00 Job 'j-2310254a9c154b6ea63e4459c749ea05': send 'start'
0:00:21 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:00:26 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:00:33 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:00:41 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:00:51 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:01:03 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:01:19 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:01:38 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:02:02 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:02:32 Job 'j-2310254a9c154b6ea63e4459c749ea05': queued (progress N/A)
0:03:10 Job 'j-2310254a9c154b6ea63e4459c749ea05': running (progress N/A)
0:03:56 Job 'j-2310254a9c154b6ea63e4459c749ea05': running (progress N/A)
0:04:55 Job 'j-2310254a9c154b6ea63e4459c749ea05': running (progress N/A

<BatchJob job_id='j-2310254a9c154b6ea63e4459c749ea05'>